In [4]:
import pandas as pd

**Data has no heading, so we need to give names to the columns**

In [5]:
columns = ['stock_identifier', 'col_2', 'bid_price', 'ask_price', 
           'trade_price', 'bid_volume', 'ask_volume', 'trade_volume', 
           'update_type', 'col_10', 'date','time_seconds', 'open_price', 
           'col_14', 'condition_codes_1', 'condition_codes_2']

**As this is an extremely large file, it will be imported in chunks**

In [6]:
scanditick = pd.read_csv("/Users/xiac/Documents/Yandex/scandi_trading_project/scandi.csv", 
                         names = columns, chunksize = 50000)

**Concat the chunks to make one complete dataframe**

In [7]:
scandi = pd.concat(scanditick, ignore_index = True)

In [8]:
scandi.head(5)

,stock_identifier,col_2,bid_price,ask_price,trade_price,bid_volume,ask_volume,trade_volume,update_type,col_10,date,time_seconds,open_price,col_14,condition_codes_1,condition_codes_2
0,BBHBEAT Index,59,121.4,125.4,123.4,1234,1234,1234,2,0,20150420,28264.0,0.0,0,@1,NaN
1,BBHBEAT Index,60,121.4,125.4,123.4,1234,1234,1234,2,0,20150420,28265.0,0.0,0,@1,NaN
2,BBHBEAT Index,61,121.4,125.4,123.4,1234,1234,1234,2,0,20150420,28266.0,0.0,0,@1,NaN
3,BBHBEAT Index,62,121.4,125.4,123.4,1234,1234,1234,2,0,20150420,28267.0,0.0,0,@1,NaN
4,BBHBEAT Index,63,121.4,125.4,123.4,1234,1234,1234,2,0,20150420,28268.0,0.0,0,@1,NaN


In [9]:
scandi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13260277 entries, 0 to 13260276
Data columns (total 16 columns):
 #   Column             Dtype  
---  ------             -----  
 0   stock_identifier   object 
 1   col_2              int64  
 2   bid_price          float64
 3   ask_price          float64
 4   trade_price        float64
 5   bid_volume         int64  
 6   ask_volume         int64  
 7   trade_volume       int64  
 8   update_type        int64  
 9   col_10             int64  
 10  date               int64  
 11  time_seconds       float64
 12  open_price         float64
 13  col_14             int64  
 14  condition_codes_1  object 
 15  condition_codes_2  object 
dtypes: float64(5), int64(8), object(3)
memory usage: 1.6+ GB


**Filter the data as required**

In [10]:
scandi_sub = scandi[(scandi['condition_codes_1'].isna()) | 
                    (scandi['condition_codes_1'] == "XT")]

In [11]:
len(scandi_sub)

13122036

In [12]:
scandi_sub = scandi_sub[scandi_sub['bid_price'] <= scandi_sub['ask_price']]

In [13]:
scandi_sub = scandi_sub.drop(['col_2', 'col_10', 'col_14'], axis = 1)

**define the functions that will go through each stock and get the last_digit_ratio, as well as time_between_trades stats**

In [14]:
def last_digit_ratio(stock_identifier, column):
    #change column to a list
    num_list = scandi_sub[scandi_sub['stock_identifier'] == stock_identifier][column].tolist()
    #change values in list to strings
    string_list = [repr(n) for n in num_list]
    #take the last item in each string then convert to integer
    last_digit_list = [int(n[-1]) for n in string_list]
    #calcuate the ratio
    ratios_dict = {}
    for i in range(10):
        ratio = last_digit_list.count(i) / len(last_digit_list) 
        ratios_dict[i] = ratio
    return ratios_dict

In [15]:
#dataset is named 'scandi', and columns named 'stock_identifer' and 'update_type'
def time_difference_stats(stock_identifier):
    from statistics import mean, median
    
    data_sub = scandi_sub[(scandi_sub['stock_identifier'] == stock_identifier) & 
                      (scandi_sub['update_type'] == 1)]
    time_seconds_list = data_sub['time_seconds'].tolist()
    
    time_difference_list = [time_seconds_list[i+1] - 
                        time_seconds_list[i] for i in range(len(time_seconds_list)-1) 
                            if time_seconds_list[i+1] > time_seconds_list[i]]

    trade_time_mean = mean(time_difference_list)
    trade_time_median = median(time_difference_list)
    trade_time_max = max(time_difference_list)
    d = {'trade_time_mean': trade_time_mean, 'trade_time_median': trade_time_median,
            'trade_time_max': trade_time_max}
    return d

**Create a list that has all the stock identifiers**

In [16]:
stock_list = scandi_sub['stock_identifier'].unique()

**create the dataframe that contains time between trades stats for all stocks**

In [17]:
time_difference_results = []
for x in stock_list:
    time_difference_results.append(time_difference_stats(x))


In [18]:
time_between_trades = pd.DataFrame(index = stock_list, 
                                           data = time_difference_results,
                                          columns = ['trade_time_mean', 'trade_time_median',
                                                    'trade_time_max'])

In [19]:
time_between_trades

,trade_time_mean,trade_time_median,trade_time_max
ABB SS Equity,33.148016,17.0,490.0
ALFA SS Equity,20.590654,5.0,447.0
ASSAB SS Equity,38.005960,15.0,537.0
ATCOA SS Equity,21.254386,10.0,349.0
ATCOB SS Equity,47.186989,24.0,611.0
...,...,...,...
STL NO Equity,17.349556,7.0,310.0
SUBC NO Equity,25.567135,11.0,639.0
TEL NO Equity,35.223965,17.0,549.0
TGS NO Equity,38.514244,13.0,879.0


**create the dataframe that contains the last digits ratios for trade prices of all stocks**

In [20]:
last_digit_results = []
for x in stock_list:
    last_digit_results.append(last_digit_ratio(x, 'trade_price'))

In [21]:
last_digit_ratio_allstocks = pd.DataFrame(index = stock_list, 
                                          data = last_digit_results,
                                         columns = [0,1,2,3,4,5,6,7,8,9])

In [22]:
last_digit_ratio_allstocks

,0,1,2,3,4,5,6,7,8,9
ABB SS Equity,0.122384,0.083067,0.076360,0.066543,0.072649,0.086368,0.094895,0.119809,0.136289,0.141635
ALFA SS Equity,0.133952,0.100751,0.109491,0.078723,0.085023,0.100570,0.087992,0.100814,0.105959,0.096725
ASSAB SS Equity,0.487347,0.000000,0.000000,0.000000,0.000000,0.512653,0.000000,0.000000,0.000000,0.000000
ATCOA SS Equity,0.112650,0.094883,0.094601,0.097079,0.093635,0.104812,0.099675,0.098858,0.099194,0.104613
ATCOB SS Equity,0.111708,0.098817,0.089008,0.107683,0.114446,0.111038,0.109922,0.089488,0.081329,0.086560
...,...,...,...,...,...,...,...,...,...,...
STL NO Equity,0.121688,0.106647,0.108597,0.091521,0.105329,0.095023,0.081767,0.078860,0.098907,0.111662
SUBC NO Equity,0.054593,0.061433,0.060529,0.059186,0.055094,0.526647,0.047617,0.048352,0.038395,0.048155
TEL NO Equity,0.116363,0.109932,0.118022,0.114238,0.089701,0.077923,0.082386,0.088445,0.103344,0.099647
TGS NO Equity,0.189526,0.123194,0.125398,0.098117,0.079326,0.077857,0.066944,0.088034,0.077585,0.074020


In [23]:
with pd.ExcelWriter('scandi_output.xlsx') as writer:
    time_between_trades.to_excel(writer, sheet_name = 'time_between_trades')
    last_digit_ratio_allstocks.to_excel(writer, sheet_name = "last_digit_ratio_allstocks")